<a href="https://colab.research.google.com/github/oguzhan-serttas/Assembly-80x86/blob/main/deneme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random

In [13]:
!pip install datasets==2.9.0

In [16]:
from datasets import load_dataset
dataset = load_dataset("emotion")

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [19]:
train = dataset['train']
val = dataset['validation']
test = dataset['test']

In [20]:
def get_tweet(data):
  tweets=[x['text'] for x in data]
  labels=[x['label'] for x in data]
  return tweets, labels

In [21]:
tweets, labels = get_tweet(train)

In [22]:
tweets[4], labels[4]

('i am feeling grouchy', 3)

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [24]:
tokenizer=Tokenizer(num_words=1000)
tokenizer.fit_on_texts(tweets)

In [28]:
tokenizer.texts_to_sequences([tweets[0]])

[[1, 138, 2, 678]]

In [26]:
tweets[4]

'i am feeling grouchy'

In [31]:
maxlen = 50
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [32]:
def get_sequences(tokenizer, tweets):
  sequences=tokenizer.texts_to_sequences(tweets)
  padded=pad_sequences(sequences, truncating='post', padding='post', maxlen=maxlen)
  return padded

In [33]:
padded_train_seq=get_sequences(tokenizer, tweets)

In [34]:
padded_train_seq[0]

array([  1, 138,   2, 678,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [52]:
classes = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
print(classes)
# sadness, joy, love, fear, anger, surprise

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [53]:
class_to_index=dict((c,i) for i,c in enumerate(classes))
index_to_class=dict((i,c) for c,i in class_to_index.items())

In [54]:
class_to_index

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [55]:
index_to_class

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [56]:
ind_to_name = lambda labels:np.array([index_to_class.get(x) for x in labels])

In [62]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(1000,16,input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(6, activation = 'softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 16)            16000     
                                                                 
 bidirectional (Bidirectiona  (None, 50, 40)           5920      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 40)               9760      
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 246       
                                                                 
Total params: 31,926
Trainable params: 31,926
Non-trainable params: 0
_________________________________________________________________
